### An implementation of a Latent Factor Model for Amazon Ratings Prediction

While not directly related to biostatistics, the purpose of this notebook is to provide a demonstration of my level of programming competence, as well as show that I have some experience in implementing algorithms from scratch. Since my coursework represents a measure of my mathematical abilities, I wanted to supplement my application by providing a measure of my programming abilities.

#### A description of the problem and the algorithm
The objective and algorithm come from a graduate level computer science course I am enrolled in at UC San Diego, CSE 258: Recommender Systems and Web Mining, from Fall 2018. Our class was tasked to find an algorithm to predict Amazon review ratings from the features within the reviews themselves, such as the text corprus, and the average ratings by users and items. While multiple algorithms were introduced for this purpose, the implementation here is called a Latent Factor Model, in which "latent factors" are randomly generated, then iteratively solved for using a procedure called Alternating Least Squares. I provide a high level overview, though an excellent overfiew of the technique can be found here: https://datajobs.com/data-science-repo/Recommender-Systems-[Netflix].pdf

The algorithm recieved considerable attention after a version of it was used in order to solve the famous Netflix Problem, as described in the preceeding document. A simplified version of the algorithm is applied here, to predict the rating in an Amazon product review.

#### The data
Each data point is a web-scraped product review from Amazon. It consists of a text corprus that makes up the review itself, and some additional features, such as item category and time of review posting. Most importantly, it contains a `ReviewerID` and an `ItemID`, and the `rating` given by a user, to the item, in their review. A head of the dataframe can be seen below.



### The objective
The task is to predict the rating given by a particular user to a particular item, in essence, trying to predict whether a user would like an item, for the greater purpose of recommending that item to the user. Naturally, this has nothing to do with biostatistics, but since the topic of data modeling and prediction is important, I feel that this demonstration is pertinent.

Let $r_{u,i}$ denote the rating given to item $i$ by user $u$. Let $N$ be the number of examples, and $Train$ indicate the set of user, item pairs within the dataset.

The objective function is to minimize the Mean Squared Error, given by $$\frac{1}{N}\sum_{i, u \in Train} (r_{u,i} - \hat{r}_{u,i})^2$$

where $\hat{r}_{u,i}$ is the predicted rating user $u$ gives to item $i$.

### The model

One simple approach to this problem is to model the rating as a function of the global average rating, the average rating given by a user $u$ and the average rating given to item $i$. This model takes on the form
$$ r_{u, i} = \alpha + \beta_i + \beta_u$$
where $\alpha$ is the global average rating, $\beta_i$ is the difference in the average rating conferred to item $i$ from the global average $\alpha$. $\beta_u$ is similar to $\beta_i$, but for users.

While this model can achieve surprisingly good performance, it can be crude.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_rows', 500)
from sklearn.model_selection import train_test_split
from scipy.stats import uniform
from random import randint
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy.sparse import hstack, csr_matrix
from scipy import sparse

In [2]:
def openingdata(f):
    for l in open(f):
        yield eval(l)
        
data = list(openingdata('train.json'))
df = pd.io.json.json_normalize(data)

test = pd.read_csv('pairs_Rating.txt')
test['reviewerID'], test['itemID'] = test['reviewerID-itemID'].str.split('-').str
test['reviewerID'] = test.reviewerID.str.strip()
test['itemID'] = test.itemID.str.strip()



joined_data = df.append(test, ignore_index=True, sort = False)

In [18]:
df.head(5)

,categories,categoryID,helpful.nHelpful,helpful.outOf,itemID,price,rating,reviewHash,reviewText,reviewTime,reviewerID,summary,unixReviewTime
0,"[[Clothing, Shoes & Jewelry, Women], [Clothing...",0,0,0,I402344648,NaN,4.0,R798569390,The model in this picture has them rolled up a...,"09 26, 2013",U490934656,High Waisted,1380153600
1,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",0,0,0,I697650540,NaN,4.0,R436443063,"I love the look of this bra, it is what I want...","02 7, 2013",U714157797,Beautiful but size runs small,1360195200
2,"[[Clothing, Shoes & Jewelry, Wedding Party Gif...",0,0,0,I464613034,19.99,5.0,R103439446,I am not comfortable with wearing my wedding b...,"03 16, 2014",U507366950,Great Alternative for Nurses,1394928000
3,"[[Clothing, Shoes & Jewelry, Women, Clothing, ...",0,0,0,I559560885,18.99,2.0,R486351639,Like the look of this top and really looks cut...,"03 10, 2014",U307862152,One size fits all...Questionable,1394409600
4,"[[Clothing, Shoes & Jewelry, Women, Plus-Size,...",0,1,1,I476005312,NaN,5.0,R508664275,I'm quite small and the XS fits me like a regu...,"07 30, 2013",U742726598,Great shirt,1375142400


In [3]:
label_enc = LabelEncoder()
users = label_enc.fit_transform(joined_data.reviewerID).reshape(-1,1)
one_hot = OneHotEncoder(sparse = True)
users = one_hot.fit_transform(users)

training_users = users[0:200000]
test_users = users[200000:214000]

label_enc = LabelEncoder()
items = label_enc.fit_transform(joined_data.itemID).reshape(-1,1)
one_hot = OneHotEncoder(sparse = True)
items = one_hot.fit_transform(items)

training_items = items[0:200000]
test_items = items[200000:214000]


training_labels = np.asarray(joined_data[0:200000].rating)

user_ratings = np.asarray(training_users.T * training_labels)
item_ratings = np.asarray(training_items.T * training_labels)
I_u = np.asarray(training_users.T.sum(axis = 1))
U_i = np.asarray(training_items.T.sum(axis = 1))

C:\Users\acros\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\acros\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

In [6]:
        #join uses and items into sparse matrix
        X = hstack((training_users, training_items))
        #define cut points and reshape labels if not already array
        user_cut = training_users.shape[1]
        end_cut = X.shape[1]
        y = np.asarray(training_labels).reshape(-1,1)
        
        #split into train and validation sets
        X_train, X_val, training_labels, y_validation = train_test_split(X, y, test_size = 0.30)
        
        #split into validation users and validation items
        training_users = X_train.T[0:user_cut].T
        training_items = X_train.T[user_cut:end_cut].T
        validation_users = X_val.T[0:user_cut].T
        validation_items = X_val.T[user_cut:end_cut].T

        y_training = training_labels.reshape(-1, 1)
        #y_train = training_labels

In [7]:
class LatentFactorModel:
    def __init__(self, C = 1, Cx = 1, Cy = 1, K = 20, random_state = 0):
        """Fit a Latent Factor Model for Rating Prediction
        
        Accepts sparse, one-hot_encoded matrices of users and items with the 
        same number N of training examples, respectively.
        
        Uses Gradient Descent in order to update parameters for the intercept,
        and the bias parameters for both users and items. Latent variable 
        approximation not yet implementented.
        
        Automatically divides the training data into a validation set of chosen
        size and computes gradient descent by checking the validation loss.
        """
        
        self.C = C
        self.Cx = Cx
        self.Cy = Cy
        self.K = K
        self.random_state = random_state
        
    def _preprocess_data(self, users, items, y, val_size = 0.20):
        """Uses the passed users, items matrices and labels in order to generate
        training and validation sets for model fitting and training. Users and 
        items should be sparse matrices with same number of training examples.
        
        val_size should be a float between 0 and 1.
        """
        
        
        #join uses and items into sparse matrix
        X = hstack((users, items))
        #define cut points and reshape labels if not already array
        user_cut = users.shape[1]
        end_cut = X.shape[1]
        y = np.asarray(y).reshape(-1,1)
        
        #split into train and validation sets
        X_train, X_val, training_labels, y_val = train_test_split(X, y, test_size = val_size)
        
        #split into validation users and validation items
        train_users = X_train.T[0:user_cut].T
        train_items = X_train.T[user_cut:end_cut].T
        val_users = X_val.T[0:user_cut].T
        val_items = X_val.T[user_cut:end_cut].T

        training_labels = training_labels.reshape(-1, 1)
        y_train = training_labels
        
        #create matrices for gradient descent
        user_ratings = np.asarray(train_users.T * y_train)
        item_ratings = np.asarray(train_items.T * y_train)
        user_ratings = user_ratings.reshape((-1,1))
        item_ratings = item_ratings.reshape((-1,1))
        I_u = np.asarray(train_users.T.sum(axis = 1))
        U_i = np.asarray(train_items.T.sum(axis = 1))
        
        #store created objects
        self.y_train = y_train
        self.y_val = y_val
        self.train_users = train_users
        self.train_items = train_items
        self.val_users = val_users
        self.val_items = val_items
        self.user_ratings = user_ratings
        self.item_ratings = item_ratings
        self.I_u = I_u
        self.U_i = U_i
    
    def _mse_loss(self, labels, predictions ):
        """Calculates the mean-squared-error loss with current parameters."""
        
        #make predictions with parameters and passed data
        predictions = np.asarray(predictions)
        #mean squared error
        sq_err = (labels - predictions)**2
        mean_sq_err = sq_err.mean()
        
        return (mean_sq_err)
    
    def _ALS_for_biases(self, users, items, y):
        

        #extract matrices and vectors for ALS
        val_size = self.val_size
        self._preprocess_data(users, items, y, val_size = val_size)
        
        #extract objects for optimization
        y_train = self.y_train
        y_val = self.y_val
        train_users = self.train_users
        train_items = self.train_items
        val_users = self.val_users
        val_items = self.val_items
        user_ratings = self.user_ratings
        item_ratings = self.item_ratings
        I_u = self.I_u
        U_i = self.U_i
        C = self.C
        max_iter = self.max_iter
        tol = self.tol
        K = self.K
        
        # initialize parameters
        self.mu_hat = 1
        self.a_hat = np.ones((train_items.shape[1], 1))
        self.b_hat = np.ones((train_users.shape[1], 1))
        N = train_users.shape[0]
        curr_iter = 0
        
        while True:
            #calculate validation loss before update
            
            
            y_pred_train = self.mu_hat + (train_users.dot(self.b_hat)) + (train_items.dot(self.a_hat))
            prev_loss = self._mse_loss(y_train, y_pred_train)
    
            #update bias terms by iteratively solving one at a time
            self.mu_hat =  (y_train.sum() - self.b_hat.sum() - self.a_hat.sum()) / N
            self.b_hat =   (user_ratings - (( I_u * self.mu_hat)  + (train_users.T * (train_items * self.a_hat)))) / (C + I_u)             
            self.a_hat =  (item_ratings - (( U_i * self.mu_hat) + (train_items.T * (train_users * self.b_hat)))) / (C + U_i)
            
            #calculate loss
            y_pred_val = self.mu_hat + (val_users.dot(self.b_hat)) + (val_items.dot(self.a_hat))
            y_pred_train = self.mu_hat + (train_users.dot(self.b_hat)) + (train_items.dot(self.a_hat))
            curr_loss = self._mse_loss(y_train, y_pred_train)
            
            #completed loop
            curr_iter += 1
            
            #Stopping criterion if loss stops decreasing or max iterations
            if (abs(prev_loss - curr_loss) < tol):
                break
            elif (curr_iter >= 100):
                break
            else:
                continue
              
        #store losses and residuals
        residuals = y_train - y_pred_train
        self.residuals = residuals.reshape((-1, 1))
        self.bias_only_train_loss = self._mse_loss(y_train, y_pred_train)
        self.bias_only_val_loss = self._mse_loss(y_val, y_pred_val)
    
    def _AlternatingLatentFactors(self, users, items, y):
        """Implements alternating least squares to find optimal parameters for latent factors"""
        

        
        
        #Fit the bias only model
        self._ALS_for_biases(users, items, y)
        
        y_train = self.y_train
        y_val = self.y_val
        train_users = self.train_users
        train_items = self.train_items
        val_users = self.val_users
        val_items = self.val_items
        user_ratings = self.user_ratings
        item_ratings = self.item_ratings
        I_u = self.I_u
        U_i = self.U_i
        #C = self.C
        Cx = self.Cx
        Cy = self.Cy
        max_iter = self.max_iter
        tol = self.tol
        K = self.K
        
        #Randomly initalize variables as non-negative values
        gamma_i = np.random.normal(scale = 1/K, size = (K, train_items.shape[1])).T
        gamma_i = csr_matrix(gamma_i)

        gamma_u = np.random.normal(scale = 1/K, size = (K, train_users.shape[1])).T
        gamma_u = csr_matrix(gamma_u)
        
        #extract residuals to create targets for latent factors and initialize iterations
        residual_matrix = train_users.T.dot(train_items.multiply(self.residuals))
        curr_iter = 0
        N = train_users.shape[0]
        
        #P = train_users.dot(gamma_u)
        #Q = train_items.dot(gamma_i)
        #gamma_dot_prod = P * Q
        #self.gamma_dot_prod = gamma_dot_prod.sum(axis = 1).reshape((-1,1))
    
        #self.mu_hat = mu_hat
        #self.a_hat = a_hat
        #self.b_hat = b_hat
        
        while True:
            #calculate validation loss before update
            
            if curr_iter == 0:
                y_pred_train = self.mu_hat + (train_users.dot(self.b_hat)) + train_items.dot(self.a_hat)
                prev_loss = self._mse_loss(y_train, y_pred_train)
            else:
                y_pred_train = self.predict(train_users, train_items)
                prev_loss = self._mse_loss(y_train, y_pred_train)


            
            #Solve for User Latent Factors
            YTY = gamma_i.T.dot(gamma_i)
            lambdaI = Cy * np.eye(N = K)
            YTY_lambdaI = YTY + lambdaI
            YTY_lambdaI = np.asarray(YTY_lambdaI)
            RY = residual_matrix.dot(gamma_i).toarray()
            self.p_u = np.linalg.solve(YTY_lambdaI, RY.T)
            #gamma_u = csr_matrix(self.p_u.T)
            
            #Solve for Item Latent Factors
            XTX = gamma_u.T.dot(gamma_u)
            lambdaI = Cx * np.eye(N = K )
            XTX_lambdaI = XTX + lambdaI
            XTX_lambdaI = np.asarray(XTX_lambdaI)
            RX = residual_matrix.T.dot(gamma_u).toarray()
            self.q_i = np.linalg.solve(XTX_lambdaI, RX.T)
    
            #compute dot product for training labels
            P = train_users.dot(self.p_u.T)
            Q = train_items.dot(self.q_i.T)
            gamma_dot_prod = P * Q
            self.gamma_dot_prod = gamma_dot_prod.sum(axis = 1).reshape((-1,1))
            #y_pred = gamma_dot_prod.sum(axis = 1)
            
            #calculate loss
            y_pred_val = self.predict(val_users, val_items)
            y_pred_train = self.predict(train_users, train_items)
            curr_loss = self._mse_loss(y_train, y_pred_train)
            
            curr_iter += 1
            #print("Iteration: %s, train_loss: %s, val loss: %s" % (curr_iter, 
            #                                                       round(curr_loss,5),
            #                                                       round(self._mse_loss(y_val, y_pred_val), 5)))
            #print("Train loss: %s " % round(self._mse_loss(y_train, y_pred_train), 5))
            #print("Val loss: %s" % round(self._mse_loss(y_val, y_pred_val), 5))
            
            #Stopping criterion if loss stops decreasing or max iterations
            if (curr_iter >= max_iter):
                break
            else:
                continue
              
        #store the fitted parameters and training/val loss
        #self.mu_hat = mu_hat
        #self.b_hat = b_hat
        #self.a_hat = a_hat
        #self.gamma_i = gamma_i
        #self.gamma_u = gamma_u
        #self.p_u = p_u
        #self.q_i = q_i
        
        self.train_loss = self._mse_loss(y_train, y_pred_train)
        
        self.validation_loss = self._mse_loss(y_val, y_pred_val)
        self.curr_iter = curr_iter
        

        

    
    def fit(self,  users, items, labels, max_iter = 100, val_size = 0.20,
           tol = 0.001):
        """Fit model with ALS."""
        
        #extract values
        self.max_iter = max_iter
        self.tol = tol
        self.val_size = val_size
        self.is_fitted = True
        
        print("Training model...")
        
        self._AlternatingLatentFactors(users, items, labels)
        
        print("Finished training after %s rounds." % self.curr_iter)
        
        self.is_fitted = True
        
        return self
    
    def predict(self, users, items):
        """Predict on a new set, users and items should have same number of encodings as
        the training/validation data
        """
        if (self.is_fitted == True):
            #load fitted parameters
            #mu_hat = self.mu_hat
            #a_hat = self.a_hat
            #b_hat = self.b_hat
            #p_u = self.p_u
            #q_i = self.q_i
            #gamma_dot_prod = self.gamma_dot_prod
            #gamma_i = self.gamma_i
            #gamma_u = self.gamma_u
            
            P = users.dot(self.p_u.T)
            Q = items.dot(self.q_i.T)
            gamma_dot_prod = P * Q
            gamma_dot_prod = gamma_dot_prod.sum(axis = 1).reshape((-1,1))
        
            predictions = self.mu_hat + users.dot(self.b_hat) + (items.dot(self.a_hat)) + gamma_dot_prod
            predictions = np.asarray(predictions)
            
            return(predictions)
        
        else:
            print("You haven't fit this model yet.")
            
    def get_scores(self):
        """Reports training and validation loss of model."""
        if (self.is_fitted == True):
            print("This model has a training loss of %s and a validation loss of %s." % 
                  (round(self.training_loss,5), round(self.validation_loss, 5)))
        else:
            print("You haven't fit this model yet.")
    
    

        

In [12]:
%%time
iterations = 10
results = []
for k in range(iterations):
    c = np.random.uniform(0.001, 10)
    cx = np.random.uniform(0.001, 100)
    cy = np.random.uniform(0.001, 100)
    k = randint(2, 50)
    latentmodel = LatentFactorModel(C = c, Cx = cx, Cy = cy, K = k)
    latentmodel = latentmodel.fit(training_users, training_items, training_labels, 
                                              max_iter = 10, val_size = 0.50, tol = 0.000001)
    score = round(latentmodel.validation_loss, 5)
    print("Validation Loss of %s achieved with C = %s, Cx = %s, Cy = %s, K = %s" % (score, c, cx, cy, k))
    results.append((score, c, cx, cy, k))

Training model...
Finished training after 10 rounds.
Validation Loss of 1.16545 achieved with C = 5.812696455764401, Cx = 92.62435692859337, Cy = 27.69774908549177, K = 5
Training model...
Finished training after 10 rounds.
Validation Loss of 1.16656 achieved with C = 7.831804492178158, Cx = 39.15516052846017, Cy = 44.96934025067759, K = 10
Training model...
Finished training after 10 rounds.
Validation Loss of 1.1629 achieved with C = 9.356335235704393, Cx = 72.43259605688469, Cy = 44.17561177067908, K = 25
Training model...
Finished training after 10 rounds.
Validation Loss of 1.16414 achieved with C = 8.298013944320866, Cx = 0.16335295029090438, Cy = 38.31840537949307, K = 19
Training model...
Finished training after 10 rounds.
Validation Loss of 1.17762 achieved with C = 3.6352548819384345, Cx = 44.88509418940052, Cy = 77.19051143951432, K = 14
Training model...
Finished training after 10 rounds.
Validation Loss of 1.18136 achieved with C = 3.6907981166552735, Cx = 17.3640998595314

In [13]:
best_Score, best_C, best_cx, best_cy, best_K = min(results, key=lambda item: item[0])
print("The best parameters are C = %s, Cx = %s, Cy = %s, K = %s, with a loss of %s" % 
      (best_C, best_cx, best_cy, best_K, round(best_Score, 5)))

The best parameters are C = 9.356335235704393, Cx = 72.43259605688469, Cy = 44.17561177067908, K = 25, with a loss of 1.1629


In [14]:
latentmodel = LatentFactorModel(C = best_C, Cx = best_cx, Cy = best_cy, K = k)
latentmodel = latentmodel.fit(training_users, training_items, y_training, 
                                              max_iter = 5, val_size = 1, tol = 0.000001)
predictions = latentmodel.predict(validation_users, validation_items)
predictions = np.clip(predictions, a_min = 1, a_max = 5)


Training model...
Finished training after 5 rounds.


In [15]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_validation, predictions)

1.1095210615871425